In [1]:
import os
import pyspark

os.environ["PYSPARK_PYTHON"]="C:\\Anaconda3\\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"]="C:\\Anaconda3\\python.exe"

myConf=pyspark.SparkConf()
#myConf=pyspark.SparkConf().set("spark.driver.bindAddress", "127.0 .0.1")
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

###### 1) DataFrame을 생성

In [2]:
data = spark\
    .read\
    .options(header='false', inferschema='true', delimiter='\n')\
    .csv(os.path.join('data', '제74주년 경찰의 날 기념식 축사.txt'))\
    .withColumnRenamed("_c0", "value")

In [3]:
data.show()

+----------------------------------+
|                             value|
+----------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|
| 국민의 안전을 위해 밤낮없이 애...|
|오늘 홍조근정훈장을 받으신 중앙...|
|           사랑하는 경찰관 여러분,|
|여러분의 헌신적 노력으로 우리의...|
| 치안의 개선은 국민의 체감으로 ...|
| 한국을 찾는 외국 관광객들도 우...|
|   올해는 ‘경찰의 날’에 맞춰 국...|
|         자랑스러운 경찰관 여러분,|
| 경찰헌장은 “나라와 겨레를 위하...|
|    대한민국 경찰은 1945년 광복...|
| 임시정부 초대 경무국장 백범 김...|
|    광복 이후 6·25전쟁에서도 경...|
|그러나 잘못도 없지는 않았습니다...|
| 지금 경찰은 과거를 돌아보며 국...|
| 검경 수사권 조정과 자치경찰제 ...|
|정부는 경찰의 근무여건을 개선하...|
| 정부는 누구도 법 위에 군림하지...|
| 경찰헌장은 따뜻한 경찰, 의로운...|
|행사를 준비하신 민갑룡 경찰청장...|
+----------------------------------+
only showing top 20 rows



###### 2) 단어로 분리해서, 출력

In [4]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="value", outputCol="words")

tokDf = tokenizer.transform(data)

In [5]:
tokDf.select("words").show()

+--------------------------------+
|                           words|
+--------------------------------+
|   [존경하는, 국민, 여러분,, ...|
|  [국민의, 안전을, 위해, 밤낮...|
|[오늘, 홍조근정훈장을, 받으신...|
|     [사랑하는, 경찰관, 여러분,]|
| [여러분의, 헌신적, 노력으로,...|
|  [치안의, 개선은, 국민의, 체...|
|  [한국을, 찾는, 외국, 관광객...|
|    [올해는, ‘경찰의, 날’에, ...|
|   [자랑스러운, 경찰관, 여러분,]|
| [경찰헌장은, “나라와, 겨레를...|
|    [대한민국, 경찰은, 1945년...|
|  [임시정부, 초대, 경무국장, ...|
|    [광복, 이후, 6·25전쟁에서...|
|  [그러나, 잘못도, 없지는, 않...|
|  [지금, 경찰은, 과거를, 돌아...|
|  [검경, 수사권, 조정과, 자치...|
| [정부는, 경찰의, 근무여건을,...|
|   [정부는, 누구도, 법, 위에,...|
|  [경찰헌장은, 따뜻한, 경찰,,...|
|  [행사를, 준비하신, 민갑룡, ...|
+--------------------------------+
only showing top 20 rows



###### 3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

In [6]:
_mystopwords=[u"먼저",u"비롯한", u"또한",u"참",u"역대", u"함께",u"등",u"그", u"대해",u"가장",u"특히", u"것이"]

In [7]:
from pyspark.ml.feature import StopWordsRemover

stop = StopWordsRemover(inputCol="words", outputCol="nostops")

stopwords=list()

for e in _mystopwords:
    stopwords.append(e)
    
stop.setStopWords(stopwords)

StopWordsRemover_aa5c80784666

In [8]:
for e in stop.getStopWords():
    print (e, end="/")

먼저/비롯한/또한/참/역대/함께/등/그/대해/가장/특히/것이/

###### 4) 불용어 제거하고, 출력

In [9]:
stopDf=stop.transform(tokDf)
stopDf.select("nostops").show()

+--------------------------------+
|                         nostops|
+--------------------------------+
|   [존경하는, 국민, 여러분,, ...|
|  [국민의, 안전을, 위해, 밤낮...|
|[오늘, 홍조근정훈장을, 받으신...|
|     [사랑하는, 경찰관, 여러분,]|
| [여러분의, 헌신적, 노력으로,...|
|  [치안의, 개선은, 국민의, 체...|
|  [한국을, 찾는, 외국, 관광객...|
|    [올해는, ‘경찰의, 날’에, ...|
|   [자랑스러운, 경찰관, 여러분,]|
| [경찰헌장은, “나라와, 겨레를...|
|    [대한민국, 경찰은, 1945년...|
|  [임시정부, 초대, 경무국장, ...|
|    [광복, 이후, 6·25전쟁에서...|
|  [그러나, 잘못도, 없지는, 않...|
|  [지금, 경찰은, 과거를, 돌아...|
|  [검경, 수사권, 조정과, 자치...|
| [정부는, 경찰의, 근무여건을,...|
|   [정부는, 누구도, 법, 위에,...|
|  [경찰헌장은, 따뜻한, 경찰,,...|
|  [행사를, 준비하신, 민갑룡, ...|
+--------------------------------+
only showing top 20 rows



###### 5) TF-IDF를 계산하고, 출력

In [10]:
from pyspark.ml.feature import HashingTF, IDF

# hashTF = HashingTF(inputCol="nostops", outputCol="hash", numFeatures=32) #  mapping indices insufficient
hashTF = HashingTF(inputCol="nostops", outputCol="hash")

hashDf = hashTF.transform(stopDf)

In [11]:
idf = IDF(inputCol="hash", outputCol="idf")

idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

In [12]:
for e in idfDf.select("hash","idf").take(10):
    print(e)

Row(hash=SparseVector(262144, {162: 1.0, 80732: 1.0, 118171: 2.0, 127225: 1.0, 142775: 1.0, 160086: 1.0, 172380: 1.0, 254275: 1.0}), idf=SparseVector(262144, {162: 2.3979, 80732: 2.3979, 118171: 2.9632, 127225: 2.3979, 142775: 1.9924, 160086: 1.2993, 172380: 2.3979, 254275: 2.3979}))
Row(hash=SparseVector(262144, {5341: 1.0, 6304: 1.0, 30732: 1.0, 39431: 1.0, 43098: 1.0, 49855: 1.0, 51468: 1.0, 63600: 1.0, 75300: 1.0, 89318: 1.0, 96799: 1.0, 110980: 1.0, 123553: 1.0, 160081: 1.0, 160086: 1.0, 167255: 1.0, 178931: 1.0, 208192: 1.0, 217323: 2.0, 257249: 1.0, 261393: 1.0}), idf=SparseVector(262144, {5341: 2.3979, 6304: 2.3979, 30732: 2.3979, 39431: 1.7047, 43098: 2.3979, 49855: 1.9924, 51468: 2.3979, 63600: 2.3979, 75300: 2.3979, 89318: 1.9924, 96799: 2.3979, 110980: 2.3979, 123553: 1.9924, 160081: 2.3979, 160086: 1.2993, 167255: 1.2993, 178931: 2.3979, 208192: 2.3979, 217323: 3.9849, 257249: 2.3979, 261393: 2.3979}))
Row(hash=SparseVector(262144, {29823: 1.0, 32228: 1.0, 36822: 1.0, 3943

###### 6) TF-IDF 컬럼을 features로 구성, 출력

In [13]:
idfDf.select("nostops", "hash").show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|nostops                                                                                                                                                                                                                                                   |hash                                                                                                                                                                                                                                 